In [108]:
import matplotlib.pyplot as plt
import numpy as np
import pyfits
import seaborn as sns
import sys
import scipy

sys.path.append("./Orders")
import Order

from scipy.interpolate import interp1d

%matplotlib qt5
plt.ion()

sns.set_context('paper')
sns.set_style('darkgrid')
sns.set_color_codes('deep')

## Bias Sec

In [21]:
def biassec(x):
    x = x.split(',')
    fnum = ((x[0])[1:]).split(':')
    snum = ((x[1])[:len(x[1])-1]).split(':')
    fnum[0] = int(fnum[0])
    fnum[1] = int(fnum[1])
    snum[0] = int(snum[0])
    snum[1] = int(snum[1])
    return fnum,snum

## Bias Trim

In [17]:
def BiasTrim(d,c,h,otype,datasec=None):
    """
    Overscan/Bias correct and Trim an IMACS chip
    """
    # bias has no significant structure, so a single median suffices, I think
    # overscan = [0:49] [2097:2145]
    def get_full_datasec(h):
        binning = h['binning']
        b1, b2 = np.array(binning.split('x')).astype('int')
        datasec = '[1:{:},1:{:}]'.format(2048/b1, 4096/b2)
        return datasec

    oxsec,oysec = biassec(h['biassec'])
    if(datasec == None):
#        dxsec,dysec = biassec(h['datasec'])
        datasec = get_full_datasec(h)
        dxsec,dysec = biassec(datasec)
    else:
        dxsec,dysec = biassec(datasec)
    if(otype=='ift'):
        oscan_data = d[(oysec[0]-1):oysec[1],(oxsec[0]-1):oxsec[1]]
        overscan = np.median(oscan_data)
        if(overscan == 0):
            overscan = zero_oscan(oscan_data)
        newdata = d[(dysec[0]-1):dysec[1],(dxsec[0]-1):dxsec[1]] - overscan
    else:
        d = d.transpose()
        oscan_data = d[oxsec[0]-1:oxsec[1],oysec[0]-1:oysec[1]]
        overscan = np.median(oscan_data)
        if(overscan == 0):
            overscan = zero_oscan(oscan_data)
        newdata = d[dxsec[0]-1:dxsec[1],dysec[0]-1:dysec[1]] - overscan
    #overscan = np.median(d[:,2048:2112])
    #newdata = d[:4095,0:2048] - overscan
    if ((c == 'c5') or (c == 'c6') or (c == 'c7') or (c == 'c8')):
        if(otype == 'iff'):
            newdata = newdata[::-1,:]
        else:
            newdata = newdata[::-1,::-1]

    return newdata

## Fix bad pixels

In [18]:
def FixBadPixels(d, xbad_pix, ybad_pix, xmin=None, xmax=None):
    if xmin is None:
        xmin = 0
    if xmax is None:
        xmax = d.shape[1]-1
    y_all = np.arange(d.shape[0])
    #counter = 0
    for i in range(xmin,xmax+1):
        idx_bad = np.where(xbad_pix == i)[0]
        if len(idx_bad)>0:
            mask = np.ones(len(y_all), dtype=bool)
            bad_y = ybad_pix[idx_bad]
            mask[bad_y] = False
            #counter = counter + 1
            # Prepare the values to interpolate by fixing values further away than the edges to 
            # the median of the ten closest points to them:
            xinterp = np.append(y_all[mask],y_all[-1]+1)
            yinterp = np.append(d[y_all[mask],i],np.median(d[-10:,i]))
            xinterp = np.append(-1,xinterp)
            yinterp = np.append(np.median(yinterp[0:10]),yinterp)
            spl = interp1d(xinterp,yinterp)
            #pyfits.PrimaryHDU(d[:,i]).writeto('before_'+str(counter)+'.fits') 
            d[bad_y,i] = spl(bad_y)
            #pyfits.PrimaryHDU(d[:,i]).writeto('after_'+str(counter)+'.fits')
            #sys.exit()
    return d

## Test noise

In [53]:
def TestNoiseThreshold(d, x_new, y, step, span):
    """
    Test if spectrum is above noise threshold for tracing algorithm
    """
    x = int(x_new)
    idx_spec = range(np.max([x-span,0]), np.min([x+span,d.shape[1]]))
    idx_left = range(np.max([x-2*span,0]), np.min([x-span,d.shape[1]]))
    idx_right = range(np.max([x+span,0]), np.min([x+2*span,d.shape[1]]))
    idx_down = np.max([y-step/2,0])
    idx_up = np.min([y+step/2+1,d.shape[0]])
    bkg = np.median(d[int(idx_down):int(idx_up),idx_left+idx_right])
    bkg_err = np.std(d[int(idx_down):int(idx_up),idx_left+idx_right])
    spec = np.sum(d[int(idx_down):int(idx_up),idx_spec] - bkg, axis=0)
    SpecAboveNoise = np.mean(spec) > 3*bkg_err
    return SpecAboveNoise

## Trace

In [2]:
def Trace(d, x0, y0, init_span=5, span=5, thres=1000, step=4, order=4,
          tr_precision=0.01,verbose=False):
    """
    Trace an object given the data and the initial position
    """    
    start_step = 10. # Initial step twice must be the largest 
    # (in order to avoid cosmic rays)! Is the most important one!
    dinit  = np.median(d[int(y0-start_step/2):int(y0+start_step/2+1),
                          int(x0-init_span):int(x0+init_span+1)], axis=0)
    x_init = x0-init_span+getCenter(dinit,precision = tr_precision)
#    X = np.arange(x0-span,x0+span+1)
#    x_init = np.sum(dinit * X) / np.sum(dinit) 
    Nx = d.shape[1]
    Ny = d.shape[0]    

    x_a = np.array([])
    y_a = np.array([])
    x_a = np.append(x_a,x_init)
    y_a = np.append(y_a,y0)

    if(verbose):
        print 'x0:',x0,'y0',y0,'(span: ',span,', step:',step,') | x_init:',x_init
    # go up
    cond = 1
    y_old=y0
    x_old=x_init
    counter = 0
    diff_pix = span/2.
    counter_thres = 5
    while(cond):
        y = y_old + step
        if ((y + step/2)>= Ny):
            cond=0
        else:
            xold_i = int(x_old)
            # X = np.arange(xold_i-span,xold_i+span+1)
            dat  = np.median(d[int(y-step/2):int(y+step/2+1),
                               int(xold_i-span):int(xold_i+span+1)], axis=0)
            x_new = xold_i-span+getCenter(dat,precision = tr_precision)
            SpecAboveNoise = TestNoiseThreshold(d, x_new, y, step, span)
            # x_new = np.sum(dat * X) / np.sum(dat) 
        #print x_new, y, np.sum(dat)
            if (counter>counter_thres):
                cond=0
            elif(abs(x_new-x_old)>diff_pix):
                counter = counter + 1
                y_old = y
            elif(not SpecAboveNoise):
                counter = counter + 1
                y_old = y
            else:
                x_a = np.append(x_a,x_new)
                y_a = np.append(y_a,y)
                y_old=y
                x_old=x_new
                counter = 0

    # go down
    # go up
    cond = 1
    y_old=y0
    x_old=x_init
    counter = 0
    while(cond):
        y = y_old - step
        if ((y - step/2) < 0):
            cond=0
        else:
            xold_i = int(x_old)
            # X = np.arange(xold_i-span,xold_i+span+1)
            dat  = np.median(d[int(y-step/2):int(y+step/2+1),
                               int(xold_i-span):int(xold_i+span+1)], axis=0)
            # x_new = np.sum(dat * X) / np.sum(dat) 
            x_new = xold_i-span+getCenter(dat,precision = tr_precision)
            SpecAboveNoise = TestNoiseThreshold(d, x_new, y, step, span)
            if (counter>counter_thres):
                cond=0
            elif(abs(x_new-x_old)>diff_pix):
                counter = counter + 1
                y_old = y
            elif(not SpecAboveNoise):
                counter = counter + 1
                y_old = y
            else:
                x_a = np.append(x_a,x_new)
                y_a = np.append(y_a,y)
                y_old=y
                x_old=x_new
                counter = 0
    # now we have the arrays with x,y. sort them and fit them with a polynomial
    I = np.argsort(y_a)
    y_a = y_a[I]
    x_a = x_a[I]
    coeffs = scipy.polyfit(y_a, x_a, order)
    return coeffs, x_a, y_a

## Get Center

In [51]:
def getCenter(y,precision = 0.1, sigma_temp = np.sqrt(0.5)):
    return Order.CCFCenter(y.astype('double'),len(y),precision,sigma_temp)

## Fix Bad Pixels

In [105]:
def FixBadPixels(d, xbad_pix, ybad_pix, xmin=None, xmax=None):
    if xmin is None:
        xmin = 0
    if xmax is None:
        xmax = d.shape[1]-1
    y_all = np.arange(d.shape[0])
    #counter = 0
    for i in range(xmin,xmax+1):
        idx_bad = np.where(xbad_pix == i)[0]
        if len(idx_bad)>0:
            mask = np.ones(len(y_all), dtype=bool)
            bad_y = ybad_pix[idx_bad]
            mask[bad_y] = False
            #counter = counter + 1
            # Prepare the values to interpolate by fixing values further away than the edges to 
            # the median of the ten closest points to them:
            xinterp = np.append(y_all[mask],y_all[-1]+1)
            yinterp = np.append(d[y_all[mask],i],np.median(d[-10:,i]))
            xinterp = np.append(-1,xinterp)
            yinterp = np.append(np.median(yinterp[0:10]),yinterp)
            spl = interp1d(xinterp,yinterp)
            #pyfits.PrimaryHDU(d[:,i]).writeto('before_'+str(counter)+'.fits') 
            d[bad_y,i] = spl(bad_y)
            #pyfits.PrimaryHDU(d[:,i]).writeto('after_'+str(counter)+'.fits')
    #sys.exit()
    return d

## Run

In [109]:
slitwidth = 55
tr_fit_order=2
tr_precision=0.01
#f = sci[i]
#c = O[obj].chip
#xbad_pix = xbad_pixels[c]
#ybad_pix = ybad_pixels[c]
xbad_pix = pyfits.getdata('./test_data/ut150224/x_bad_pixels_c8.fits')                                                         
ybad_pix = pyfits.getdata('./test_data/ut150224/y_bad_pixels_c8.fits')
fpath = './test_data/ut150224/ift0341c8.fits'
d,h = pyfits.getdata(fpath, header=True)
d = BiasTrim(d, 'c8', h, 'ift')
d = FixBadPixels(d,xbad_pix,ybad_pix)  
#ronoise = h['ENOISE']
#gain    = h['EGAIN']
x0 = 307.68 #O[obj].x0
y0 = 871.837097 #O[obj].y0
#if(otype == 'ift'):
#   d = np.rot90(d,1)
#   x0 = O[obj].y0
#   y0 = d.shape[0]-O[obj].x0
#trimmed_image[obj] = d
print 'Trace Inputs: d: {}, x0: {}, y0: {}, slitwidth: {}, init_span: {}, tr_fit_order: {}, tr_prec: {}'.format(                                   d, x0, y0, slitwidth, int((slitwidth/2.)+0.5), tr_fit_order, tr_precision)
tc,x_a,y_a = Trace(d,x0,y0,init_span=int((slitwidth/2.)+0.5),
                                order=tr_fit_order,verbose=True,
                                tr_precision=tr_precision)
print '...done!'
# Now check for outlier points. They must go out, as they later go to the FullTrace algorithm which does not
# detect outliers. First, get the predicted x values and substract with observed:
x_predicted = np.polyval(tc,y_a)
res = (np.array(x_a)-x_predicted)
# Calculate robust sigma:
mad = np.median(np.abs(res-np.median(res)))
sigma = 1.4826*mad

Trace Inputs: d: [[ 85.2630597    9.96153846  13.55102041 ...   7.76923077   2.25
   10.55172414]
 [ 85.0261194    9.92307692  13.60204082 ...   7.53846154   0.5
   10.60344828]
 [ 84.7891791    9.88461538  13.65306122 ...   7.30769231  -1.25
   10.65517241]
 ...
 [148.78607524 152.75971731 120.81110475 ...   2.4375      12.
    8.        ]
 [148.85738349 152.83981154 120.87406983 ...   2.125       10.
    6.83333333]
 [148.92869175 152.91990577 120.93703492 ...   1.8125       8.
    5.66666667]], x0: 307.68, y0: 871.837097, slitwidth: 55, init_span: 28, tr_fit_order: 2, tr_prec: 0.01
x0: 307.68 y0 871.837097 (span:  5 , step: 4 ) | x_init: 315.97
...done!
